<a href="https://colab.research.google.com/github/prithvivinodnair/Data-Visualization/blob/main/HW3_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from bokeh.plotting import figure, output_file, save
from bokeh.models import ColumnDataSource, HoverTool, RangeSlider, CustomJS
from bokeh.layouts import column
from bokeh.palettes import Category20
from bokeh.transform import factor_cmap

# Load and clean data
df = pd.read_csv("DV.csv")
df = df[['targtype1_txt', 'nkill', 'nwound', 'iyear']].dropna()
df = df[(df['nkill'] >= 0) & (df['nwound'] >= 0)]
df['casualties'] = df['nkill'] + df['nwound']
df = df[df['casualties'] <= 500]

# Get unique target types and full year range
target_types = df['targtype1_txt'].unique().tolist()
min_year, max_year = int(df['iyear'].min()), int(df['iyear'].max())

# Initial data (first 10 years)
initial_df = df[(df['iyear'] >= min_year) & (df['iyear'] <= min_year + 10)]
source = ColumnDataSource(initial_df)
full_source = ColumnDataSource(df)

# Figure setup
p = figure(title=f"Target Types and Casualties ({min_year}–{min_year+10})",
           x_axis_label="Number Killed", y_axis_label="Number Wounded",
           tools="pan,wheel_zoom,reset")

color_map = factor_cmap('targtype1_txt', palette=Category20[20], factors=target_types)

# Use scatter instead of circle (Bokeh 3.4+)
p.scatter('nkill', 'nwound',
          size='casualties',
          source=source,
          fill_alpha=0.6,
          color=color_map,
          legend_field='targtype1_txt')

# Hover
hover = HoverTool(tooltips=[
    ("Target", "@targtype1_txt"),
    ("Killed", "@nkill"),
    ("Wounded", "@nwound"),
    ("Year", "@iyear")
])
p.add_tools(hover)

# Legend
p.legend.location = "top_left"
p.legend.label_text_font_size = "8pt"
p.legend.click_policy = "hide"

# RangeSlider for year filtering
slider = RangeSlider(start=min_year, end=max_year,
                     value=(min_year, min_year + 10),
                     step=1, title="Year Range")

# JavaScript callback
callback = CustomJS(args=dict(source=source,
                              full=full_source,
                              slider=slider),
                    code="""
    const data = source.data;
    const all = full.data;
    const [start, end] = slider.value;

    let nkill = [], nwound = [], targ = [], cas = [], year = [];

    for (let i = 0; i < all['iyear'].length; i++) {
        if (all['iyear'][i] >= start && all['iyear'][i] <= end) {
            nkill.push(all['nkill'][i]);
            nwound.push(all['nwound'][i]);
            targ.push(all['targtype1_txt'][i]);
            cas.push(all['casualties'][i]);
            year.push(all['iyear'][i]);
        }
    }

    data['nkill'] = nkill;
    data['nwound'] = nwound;
    data['targtype1_txt'] = targ;
    data['casualties'] = cas;
    data['iyear'] = year;
    source.change.emit();
""")

slider.js_on_change('value', callback)

layout = column(p, slider)

# Save to HTML
output_file("bokeh_target_casualties.html")
save(layout)

print(" HTML file generated: 'bokeh_target_casualties.html'")


 HTML file generated: 'bokeh_target_casualties.html'
